In [1]:
import pandas as pd, os, gc
import numpy as np
import math
import os
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.express as px
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from typing import List 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
path = '/content/drive/My Drive/10Acad/Train.csv'
path_test = '/content/drive/My Drive/10Acad/Test.csv'

In [4]:
train =pd.read_csv(path, parse_dates=['join_date'])
test = pd.read_csv(path_test, parse_dates=['join_date'])
    
    
#Store Products ID in Multidimensional array
products =train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
products_t =test[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
    
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products_t, var_name = "PCODE", value_name="Label" )
    
    #Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE']
test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']
test.drop('Label', inplace=True, axis=1) 

In [5]:
train.fillna('2018-01-05', inplace=True)
test.fillna('2018-01-05', inplace=True)

In [6]:
#Computing the duration  of the client in the insurance company
#converting the 'Date' column  to date formate
train["join_date"] = pd.to_datetime(train["join_date"],format = "%Y-%m-%d")
test["join_date"] = pd.to_datetime(test["join_date"],format = "%Y-%m-%d")
today = datetime.date.today()
    
train["duration"] = today.year - train["join_date"].dt.year
#Computing the age  of the client
train["age"] = today.year - train["birth_year"]

test["duration"] = today.year - test["join_date"].dt.year
#Computing the age  of the client
test["age"] = today.year - test["birth_year"]

In [7]:
# function to preprocess our data from train models
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
scaler = StandardScaler()
def preprocessing_data(data):

    # Convert the following numerical labels from integer to float
    float_array = data[["age", "duration","birth_year"]].values.astype(float)
    
    # categorical features to be onverted to One Hot Encoding
    categ = [
    'branch_code','occupation_category_code','marital_status'
    ]
    
    # One Hot Encoding conversion
    data = pd.get_dummies(data, prefix_sep="_", columns=categ)
    
    # Binary Encoding : sex
    data["IsMale"] = np.where(data.sex == 'M', 1, 0)
    
    # Label Encoder conversion
    data["occupation_code"] = le.fit_transform(data["occupation_code"])
    data["PCODE"] = le.fit_transform(data["PCODE"])
    data["ID"] = le.fit_transform(data["ID"])
    
    # drop sex column
    data = data.drop(["sex"], axis=1)
    
    # Select numerical columns
    numeric_columns = ["age","duration","birth_year"]
    data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
    
    return data

In [8]:
train_scaled = preprocessing_data(train)
test_scaled = preprocessing_data(test)

In [9]:
print(train_scaled.shape)
print(test_scaled.shape)

(611772, 39)
(210000, 38)


In [10]:
'''def get_submission_file(model, filename):
    # converting to the matrix
    test_X = preprocessed_test_data.as_matrix().astype('float')
    # make predictions
    predictions = model.predict(test_X)
    # submission dataframe
    df_submission = pd.DataFrame({'ID X PCODE': preprocessed_test_data["ID X PCODE"], 'Label' : predictions})
    # submission file
    submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
    submission_file_path = os.path.join(submission_data_path, filename)
    # write to the file
    df_submission.to_csv(submission_file_path, index=False)'''

'def get_submission_file(model, filename):\n    # converting to the matrix\n    test_X = preprocessed_test_data.as_matrix().astype(\'float\')\n    # make predictions\n    predictions = model.predict(test_X)\n    # submission dataframe\n    df_submission = pd.DataFrame({\'ID X PCODE\': preprocessed_test_data["ID X PCODE"], \'Label\' : predictions})\n    # submission file\n    submission_data_path = os.path.join(os.path.pardir, \'data\', \'external\')\n    submission_file_path = os.path.join(submission_data_path, filename)\n    # write to the file\n    df_submission.to_csv(submission_file_path, index=False)'

In [17]:
X = train_scaled.drop(columns=['join_date', 'ID X PCODE','birth_year', 'Label'])
y = train_scaled['Label']
sampled_test = test_scaled.drop(columns=['join_date', 'birth_year','ID X PCODE'])

In [12]:
#LightGBM
import lightgbm as lgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1234)
SEARCH_PARAMS = {'learning_rate': 0.4,
                'max_depth': 15,
                'num_leaves': 32,
                'feature_fraction': 0.8,
                'subsample': 0.2}

FIXED_PARAMS={'objective': 'binary',
             'metric': 'auc',
             'is_unbalance':True,
             'bagging_freq':5,
             'boosting':'dart',
             'num_boost_round':300,
             'early_stopping_rounds':30}
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)
params = {'metric':FIXED_PARAMS['metric'],
             'objective':FIXED_PARAMS['objective']}
model = lgb.train(params, train_data,                     
                     valid_sets=[valid_data],
                     num_boost_round=FIXED_PARAMS['num_boost_round'],
                     early_stopping_rounds=FIXED_PARAMS['early_stopping_rounds'],
                     valid_names=['valid'])

score = model.best_score['valid']['auc']
score

#get_submission_file(model, "lightgbm_submission5.csv")

[1]	valid's auc: 0.932586
Training until validation scores don't improve for 30 rounds.
[2]	valid's auc: 0.949224
[3]	valid's auc: 0.948716
[4]	valid's auc: 0.950446
[5]	valid's auc: 0.950827
[6]	valid's auc: 0.957779
[7]	valid's auc: 0.958199
[8]	valid's auc: 0.958221
[9]	valid's auc: 0.959992
[10]	valid's auc: 0.96037
[11]	valid's auc: 0.961388
[12]	valid's auc: 0.961773
[13]	valid's auc: 0.961662
[14]	valid's auc: 0.96178
[15]	valid's auc: 0.9623
[16]	valid's auc: 0.963788
[17]	valid's auc: 0.964349
[18]	valid's auc: 0.964577
[19]	valid's auc: 0.964561
[20]	valid's auc: 0.965007
[21]	valid's auc: 0.965105
[22]	valid's auc: 0.96569
[23]	valid's auc: 0.96598
[24]	valid's auc: 0.966181
[25]	valid's auc: 0.966354
[26]	valid's auc: 0.966978
[27]	valid's auc: 0.967762
[28]	valid's auc: 0.967991
[29]	valid's auc: 0.968302
[30]	valid's auc: 0.968732
[31]	valid's auc: 0.969036
[32]	valid's auc: 0.969269
[33]	valid's auc: 0.969397
[34]	valid's auc: 0.969524
[35]	valid's auc: 0.969792
[36]	val

0.9757050830669202

In [25]:
test_X = sampled_test.values.astype('float')
pred = model.predict(test_X)

In [19]:
#pred = clf.predict(sampled_test)
#submission = pd.DataFrame({'ID X PCODE' : test['ID X PCODE'], 'Label':pred.index})
submission = pd.DataFrame({'ID X PCODE':test['ID X PCODE'],'Label':pred})

print('Pred_0: {}'.format(submission[submission['Label']==0].count()))
print('----------------------------------------------------------')
print('Pred_1: {}'.format(submission[submission['Label']==1].count()))

Pred_0: ID X PCODE    0
Label         0
dtype: int64
----------------------------------------------------------
Pred_1: ID X PCODE    0
Label         0
dtype: int64


In [20]:
submission.to_csv('/content/drive/My Drive/10Acad/data/submission23.csv', index=False)